
# LLM Chatbot Behavior With Pretrained data (No document or prompt)

### **Goals for the MEPO LLM Chat Bot**
This notebook is designed to explore the behavior and responses of an LLM chatbot, specifically when it operates without a document as context and without a prompt guiding its behavior. Through this exercise, you will critically analyze how these AI systems function and consider the implications of their usage.

### **Context and Importance**

As AI systems become increasingly integrated into our daily lives, it's crucial to understand how they operate and the implications of their use. This notebook encourages you to critically think about the role of AI chatbots and what it means to engage with them meaningfully.

###  **Dangers of AI Reliance**

While AI chatbots can be incredibly useful, they also come with potential pitfalls. Over-reliance on these systems can introduce risks such as:
- **Bias:** AI models can perpetuate or amplify existing biases present in their training data.
- **Lack of Transparency:** Understanding how AI systems make decisions can be challenging, leading to a "black box" problem.
- **Amplification of Inequalities:** AI may inadvertently reinforce societal inequalities.

###  **Importance of Attention and Critical Thinking**

Human attention and critical thinking are paramount when using AI systems. Relying too heavily on AI can lead to decreased engagement and a decline in essential analytical skills. This notebook will emphasize the importance of maintaining a balance between AI assistance and human judgment.

###  **Responsible AI Usage**

This section addresses the ethical considerations and guidelines for responsible AI adoption. It will help you answer the question, "How should I use these AI systems?" by providing a framework for ethical and mindful interaction with AI.

### **Choice of LLM (Open Data)**

The notebook will also explore the rationale behind choosing an open-data LLM. Open-data models promote transparency, accessibility, and the potential for community-driven development, making them a valuable tool for education and innovation.

---

# Purpose of the Notebook

The primary purpose of this notebook is to guide you through the necessary steps to set up and interact with the LLaMA-2 model. By following along, you will gain insights into how these models work, and how they respond in the absence of document-based context. This hands-on experience will help in forming design solutions and understanding the broader implications of AI systems.



## Install Required Libraries
We need to install the necessary libraries for Tranformers. Additionally, we'll install a dependency for our user interface using gradio.

In [1]:

!pip install bitsandbytes -q --upgrade
!pip install accelerate transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

## Import Required Libraries
Next, we'll import the necessary libraries for tokenization, model setup, and text generation.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
import gradio as gr


## Define Model and Tokenizer
We'll define the model name and the authentication token required to access the LLaMA-2 model from Hugging Face.

In [4]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
token_file = open("HF_TOKEN.txt")
auth_token = token_file.readline().strip();


tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='./model/', token=auth_token)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Load the Model
Now, we'll load the LLaMA-2 model using the previously defined name and authentication token. We'll also set some model parameters.

In [5]:
# Load the LLaMA-2 model using the previously defined name and authentication token. We'll also set some model parameters.
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir='./model/',
                                             token=auth_token,
                                             torch_dtype= torch.float16,
                                             rope_scaling={"type": "dynamic", "factor": 2},
                                             load_in_8bit=True)




config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

#Gradio
Using gradio for inputs and outputs of llms without loaded doc or prompts.

In [6]:
def generate_response(user_input):
  prompt = f"### User:{user_input} ### Assistant:"
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
  output = model.generate(**inputs, streamer=streamer, use_cache=True, max_new_tokens=500)
  output_text = tokenizer.decode(output[0], skip_special_tokens=True)
  # Split the output and return only the assistant's response
  assistant_response = output_text.split("### Assistant:")[-1].strip()
  return assistant_response

create a Gradio interface:

In [7]:
demo = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, label="Enter your question:"),
    outputs=gr.Textbox(label="Model Response"),
)

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f9db04e32265edbbc7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


During the 1910s, telemedicine technology was still in its infancy, and the available technology was largely limited to telephone and telegraph systems. However, there were some innovative solutions that could have been implemented to combat the Spanish Flu pandemic using the technology available at the time. Here are a few examples:

1. Telephone-based consultations: Doctors could have used telephone lines to conduct remote consultations with patients, especially in areas where in-person visits were not feasible due to the widespread nature of the pandemic. This would have allowed doctors to assess patients' symptoms and provide medical advice over the phone.
2. Telegraph-based reporting: Hospitals and medical facilities could have used telegraph systems to report on the number of cases and deaths caused by the pandemic. This would have allowed health officials to track the spread of the virus and allocate resources more effectively.
3. Radio broadcasts: Radio broadcasts could have be

In [10]:
generate_response("How could a telemedicine system have been implemented using the technology available during the 1910s to combat the Spanish Flu pandemic?")


During the 1910s, telemedicine as we know it today did not exist. However, there were some early precursors to telemedicine that could have been used to combat the Spanish Flu pandemic. Here are a few examples:

1. Telegraph-based consultations: Doctors could have used telegraph machines to communicate with each other and with patients remotely. This would have allowed them to ask questions, provide advice, and even conduct basic diagnoses over long distances.
2. Wireless telegraphy: Wireless telegraphy, also known as radio communication, was becoming more widely available during the 1910s. Doctors could have used wireless telegraphy to send messages to each other and to patients, particularly in areas where traditional telegraph lines were not available.
3. Photography: Photography was already widely used in the medical field during the 1910s, and could have been used to remotely examine patients. Doctors could have taken photographs of patients' symptoms and shared them with other do

"During the 1910s, telemedicine as we know it today did not exist. However, there were some early precursors to telemedicine that could have been used to combat the Spanish Flu pandemic. Here are a few examples:\n\n1. Telegraph-based consultations: Doctors could have used telegraph machines to communicate with each other and with patients remotely. This would have allowed them to ask questions, provide advice, and even conduct basic diagnoses over long distances.\n2. Wireless telegraphy: Wireless telegraphy, also known as radio communication, was becoming more widely available during the 1910s. Doctors could have used wireless telegraphy to send messages to each other and to patients, particularly in areas where traditional telegraph lines were not available.\n3. Photography: Photography was already widely used in the medical field during the 1910s, and could have been used to remotely examine patients. Doctors could have taken photographs of patients' symptoms and shared them with oth

In [11]:
generate_response("What affordable and sustainable housing solutions could have been developed during the Great Depression to address widespread homelessness?")


During the Great Depression, many affordable and sustainable housing solutions were developed to address widespread homelessness. Here are some examples: 1. Community Land Trusts: Community Land Trusts (CLTs) are nonprofit organizations that hold land for the benefit of the community, rather than private individuals. This approach allows for affordable housing to be built and maintained without the profit motive. CLTs can also provide a source of affordable land for low-income families to build their own homes. 2. Mutual Housing: Mutual housing is a type of cooperative housing where residents own and control the property, and make decisions about its management and maintenance. This approach can provide affordable housing while also fostering a sense of community and collective ownership. 3. Tiny Houses: Tiny houses are small, portable homes that are often built on wheels. They can be used as a solution for affordable housing, as they are often cheaper to build and maintain than tradit

'During the Great Depression, many affordable and sustainable housing solutions were developed to address widespread homelessness. Here are some examples: 1. Community Land Trusts: Community Land Trusts (CLTs) are nonprofit organizations that hold land for the benefit of the community, rather than private individuals. This approach allows for affordable housing to be built and maintained without the profit motive. CLTs can also provide a source of affordable land for low-income families to build their own homes. 2. Mutual Housing: Mutual housing is a type of cooperative housing where residents own and control the property, and make decisions about its management and maintenance. This approach can provide affordable housing while also fostering a sense of community and collective ownership. 3. Tiny Houses: Tiny houses are small, portable homes that are often built on wheels. They can be used as a solution for affordable housing, as they are often cheaper to build and maintain than tradi

In [12]:
generate_response("How might renewable energy technologies like solar or wind power have been developed in the 1970s to address the energy crisis?")


Renewable energy technologies like solar or wind power were not yet developed in the 1970s to address the energy crisis. However, there were other technologies and strategies that were developed and implemented during that time to address the energy crisis. Some of these include:

1. Conservation: One of the most effective ways to address the energy crisis was to reduce energy consumption through conservation. This involved implementing energy-efficient technologies and practices, such as using energy-efficient appliances, insulating homes and buildings, and reducing energy usage in industries.
2. Nuclear power: In the 1970s, nuclear power was seen as a potential solution to the energy crisis. Nuclear power plants were built to provide a reliable and clean source of energy. However, the accidents at Three Mile Island in 1979 and Chernobyl in 1986 raised concerns about the safety of nuclear power.
3. Coal: Coal was also seen as a potential solution to the energy crisis in the 1970s. Coa

'Renewable energy technologies like solar or wind power were not yet developed in the 1970s to address the energy crisis. However, there were other technologies and strategies that were developed and implemented during that time to address the energy crisis. Some of these include:\n\n1. Conservation: One of the most effective ways to address the energy crisis was to reduce energy consumption through conservation. This involved implementing energy-efficient technologies and practices, such as using energy-efficient appliances, insulating homes and buildings, and reducing energy usage in industries.\n2. Nuclear power: In the 1970s, nuclear power was seen as a potential solution to the energy crisis. Nuclear power plants were built to provide a reliable and clean source of energy. However, the accidents at Three Mile Island in 1979 and Chernobyl in 1986 raised concerns about the safety of nuclear power.\n3. Coal: Coal was also seen as a potential solution to the energy crisis in the 1970s